### Student Information
Name: 黃詠家 (Yung-Chia Huang)

Student ID: 112061520

GitHub ID: 102374298

---

### Instructions

1. First: do the **take home** exercises in the [DM2024-Lab1-Master](https://github.com/didiersalazar/DM2024-Lab1-Master.git). You may need to copy some cells from the Lab notebook to this notebook. __This part is worth 20% of your grade.__


2. Second: follow the same process from the [DM2024-Lab1-Master](https://github.com/didiersalazar/DM2024-Lab1-Master.git) on **the new dataset**. You don't need to explain all details as we did (some **minimal comments** explaining your code are useful though).  __This part is worth 30% of your grade.__
    - Download the [the new dataset](https://huggingface.co/datasets/Senem/Nostalgic_Sentiment_Analysis_of_YouTube_Comments_Data). The dataset contains a `sentiment` and `comment` columns, with the sentiment labels being: 'nostalgia' and 'not nostalgia'. Read the specificiations of the dataset for background details. 
    - You are allowed to use and modify the `helper` functions in the folder of the first lab session (notice they may need modification) or create your own.


3. Third: please attempt the following tasks on **the new dataset**. __This part is worth 30% of your grade.__
    - Generate meaningful **new data visualizations**. Refer to online resources and the Data Mining textbook for inspiration and ideas. 
    - Generate **TF-IDF features** from the tokens of each text. This will generating a document matrix, however, the weights will be computed differently (using the TF-IDF value of each word per document as opposed to the word frequency). Refer to this Scikit-learn [guide](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) .
    - Implement a simple **Naive Bayes classifier** that automatically classifies the records into their categories. Use both the TF-IDF features and word frequency features to build two seperate classifiers. Note that for the TF-IDF features you might need to use other type of NB classifier different than the one in the Master Notebook. Comment on the differences.  Refer to this [article](https://hub.packtpub.com/implementing-3-naive-bayes-classifiers-in-scikit-learn/).


4. Fourth: In the lab, we applied each step really quickly just to illustrate how to work with your dataset. There are somethings that are not ideal or the most efficient/meaningful. Each dataset can be handled differently as well. What are those inefficent parts you noticed? How can you improve the Data preprocessing for these specific datasets? __This part is worth 10% of your grade.__


5. Fifth: It's hard for us to follow if your code is messy, so please **tidy up your notebook** and **add minimal comments where needed**. __This part is worth 10% of your grade.__


You can submit your homework following these guidelines: [Git Intro & How to hand your homework](https://github.com/didiersalazar/DM2024-Lab1-Master/blob/main/Git%20Intro%20%26%20How%20to%20hand%20your%20homework.ipynb). Make sure to commit and save your changes to your repository __BEFORE the deadline (October 27th 11:59 pm, Sunday)__. 

---

## The Data
Since the .csv file is open using pandas libary, we don't have to do data transform to pandas dataframe.

In [2]:
### Begin Assignment Here
import pandas as pd

df = pd.read_csv("Nostalgic_Sentiment_Analysis_of_YouTube_Comments_Data.csv")

In [19]:
# This cell is used for getting familiar with the data

df.head()
# df.tail()
df[0:2]
# df[0:2][['comment', 'sentiment']]
# df['sentiment'].value_counts()

,sentiment,comment
0,not nostalgia,He was a singer with a golden voice that I lov...
1,nostalgia,The mist beautiful voice ever I listened to hi...


In [8]:
# This cell is used for getting familiar with the data

# 1. How many rows and columns are in the dataset?
rows, columns = df.shape
print(f"1. There are {rows} rows and {columns} columns in the dataset.")

# 2. What are the column names?
print(f"2. The column names are {', '.join(df.columns)}.")

# 3. What are the data types of each column?
print("3. The data types of each column are:")
print(df.dtypes)

# 4. Are there any missing values?
missing_values = df.isnull().sum().sum()
if missing_values == 0:
    print("4. There are no missing values.")
else:
    print(f"4. There are {missing_values} missing values.")

# 5. How many unique values are in each column?
print("5. The number of unique values in each column are:")
print(df.nunique())

1. There are 1500 rows and 2 columns in the dataset.
2. The column names are sentiment, comment.
3. The data types of each column are:
sentiment    object
comment      object
dtype: object
4. There are no missing values.
5. The number of unique values in each column are:
sentiment       2
comment      1499
dtype: int64
